In [1]:
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt
#jupyter内で画像を表示
%matplotlib inline 
#別ウィンドウで画像表示
# %matplotlib qt 
import csv
import json
from ipywidgets import interact, interactive, fixed, RadioButtons
import ipywidgets as widgets
from IPython.display import display
import pathlib
import glob
import re
import copy

In [2]:
shadow = 'shadow10_2'
evaluation_path = "../../get_data/geocast_output/" + shadow + "/"
sigo_files = glob.glob(evaluation_path + "gsigo/*")
sigo_file_list = []
for file in sigo_files:
    sigo_file_list.append(file)

# print('receive log')
# print(sigo_file_list)

In [3]:
shadow = 'shadow10_2'
evaluation_path = "../../get_data/geocast_output/" + shadow + "/"
lsgo_files = glob.glob(evaluation_path + "glsgo/*")
lsgo_file_list = []
for file in lsgo_files:
    lsgo_file_list.append(file)

# print('receive log')
# print(lsgo_file_list)

# seed値リストを登録

In [4]:
seed_index = 2
node_index = 3
seed_list = []
nodenum_list = ['200', '300', '400'] ###############nodenumの形式を変更したらここはベタ打ちで変更を加える
regex = re.compile('\d+')

for path in sigo_file_list:
    for line in path.splitlines():
        match = regex.findall(line)
        if match[seed_index] not in seed_list:
            seed_list.append(match[seed_index])
            
print(seed_list)
print('simulation 試行回数',len(seed_list))

['10001', '10002', '10003', '10004', '10005', '10006', '10007', '10008', '10009', '10010', '10011', '10012', '10013', '10014', '10015', '10016', '10017', '10018', '10019', '10020', '10021', '10022', '10023', '10024', '10025', '10026', '10027', '10028', '10029', '10030', '10031', '10032', '10033', '10034', '10035', '10036', '10037', '10038', '10039', '10040', '10041', '10042', '10043', '10044', '10045', '10046', '10047', '10048', '10049', '10050', '10051', '10052', '10053', '10054', '10055', '10056', '10057', '10058', '10059', '10060', '10061', '10062', '10063', '10064', '10065', '10066', '10067', '10068', '10069', '10070', '10071', '10072', '10073', '10074', '10075', '10076', '10077', '10078', '10079', '10080', '10081', '10082', '10083', '10084', '10085', '10086', '10087', '10088', '10089', '10090', '10091', '10092', '10093', '10094', '10095', '10096', '10097', '10098', '10099']
simulation 試行回数 99


In [5]:
sigo_dict = {}
check_seed = seed_list[0]
sigo_dict[check_seed] = {}
for path in sigo_file_list:
    for line in path.splitlines():
      match = regex.findall(line) # match[2] = seed ,match[3] = nudenum
     
    if match[seed_index] == check_seed:
        sigo_dict[match[seed_index]][match[node_index]] = path
    else:#ロープ中seed値が変わるとき
        sigo_dict[match[seed_index]] = {}
        sigo_dict[match[seed_index]][match[node_index]] = path
        check_seed = match[seed_index] #seed値の更新

print(sigo_dict)

{'10001': {'200': '../../get_data/geocast_output/shadow10_2/gsigo\\gsigo-seed_10001nodenum_200.csv', '300': '../../get_data/geocast_output/shadow10_2/gsigo\\gsigo-seed_10001nodenum_300.csv', '400': '../../get_data/geocast_output/shadow10_2/gsigo\\gsigo-seed_10001nodenum_400.csv'}, '10002': {'200': '../../get_data/geocast_output/shadow10_2/gsigo\\gsigo-seed_10002nodenum_200.csv', '300': '../../get_data/geocast_output/shadow10_2/gsigo\\gsigo-seed_10002nodenum_300.csv', '400': '../../get_data/geocast_output/shadow10_2/gsigo\\gsigo-seed_10002nodenum_400.csv'}, '10003': {'200': '../../get_data/geocast_output/shadow10_2/gsigo\\gsigo-seed_10003nodenum_200.csv', '300': '../../get_data/geocast_output/shadow10_2/gsigo\\gsigo-seed_10003nodenum_300.csv', '400': '../../get_data/geocast_output/shadow10_2/gsigo\\gsigo-seed_10003nodenum_400.csv'}, '10004': {'200': '../../get_data/geocast_output/shadow10_2/gsigo\\gsigo-seed_10004nodenum_200.csv', '300': '../../get_data/geocast_output/shadow10_2/gsigo\\

In [6]:
lsgo_dict = {}
check_seed = seed_list[0]
lsgo_dict[check_seed] = {}
for path in lsgo_file_list:
    for line in path.splitlines():
      match = regex.findall(line) # match[2] = seed ,match[3] = nudenum
     
    if match[seed_index] == check_seed:
        lsgo_dict[match[seed_index]][match[node_index]] = path
    else:#ロープ中seed値が変わるとき
        lsgo_dict[match[seed_index]] = {}
        lsgo_dict[match[seed_index]][match[node_index]] = path
        check_seed = match[seed_index] #seed値の更新
        
print(lsgo_dict)

{'10001': {'200': '../../get_data/geocast_output/shadow10_2/glsgo\\glsgo-seed_10001nodenum_200.csv', '300': '../../get_data/geocast_output/shadow10_2/glsgo\\glsgo-seed_10001nodenum_300.csv', '400': '../../get_data/geocast_output/shadow10_2/glsgo\\glsgo-seed_10001nodenum_400.csv'}, '10002': {'200': '../../get_data/geocast_output/shadow10_2/glsgo\\glsgo-seed_10002nodenum_200.csv', '300': '../../get_data/geocast_output/shadow10_2/glsgo\\glsgo-seed_10002nodenum_300.csv', '400': '../../get_data/geocast_output/shadow10_2/glsgo\\glsgo-seed_10002nodenum_400.csv'}, '10003': {'200': '../../get_data/geocast_output/shadow10_2/glsgo\\glsgo-seed_10003nodenum_200.csv', '300': '../../get_data/geocast_output/shadow10_2/glsgo\\glsgo-seed_10003nodenum_300.csv', '400': '../../get_data/geocast_output/shadow10_2/glsgo\\glsgo-seed_10003nodenum_400.csv'}, '10004': {'200': '../../get_data/geocast_output/shadow10_2/glsgo\\glsgo-seed_10004nodenum_200.csv', '300': '../../get_data/geocast_output/shadow10_2/glsgo\\

In [7]:
# 列ごとの平均を返す関数
def col_average(seeds):
    new_list = []
    for seed in seeds:
        del seed[len(nodenum_list)] #1列ずつ平均を算出するうえで邪魔なseed値を消していく seedの個数 
        new_list.append(seed)
#     np.set_printoptions(precision=3, suppress=True)
    np_new_list = np.array(new_list)
    return np.mean(np_new_list, axis=0)

In [8]:
# 列ごとの平均を返す関数
def col_average_zero_delete(seeds):
    new_list = []
    for seed in seeds:
        del seed[len(nodenum_list)] #1列ずつ平均を算出するうえで邪魔なseed値を消していく seedの個数 
        new_list.append(seed)
    np_new_list = np.array(new_list)
    return np.mean(np_new_list, axis=0)

In [9]:
# packet delivery rationの平均を計算 SIGO Geocast
total_pdr_list = []
for seed in seed_list:
    pdr_list = []   # index 0: 200nodes  index 1: 300nodes index:2 400nodes
    for node in nodenum_list:
        path = sigo_dict[seed][node]
        sigo_df = pd.read_csv(path)
        pdr = sigo_df['PDR'].mean()
        pdr_list.append(pdr)
    pdr_list.append(seed)
    total_pdr_list.append(pdr_list)

# print(total_pdr_list)
print(col_average(total_pdr_list))

[0.36939126 0.49690968 0.80855221]


In [11]:
# packet delivery rationの平均を計算 LSGO Geocast
total_pdr_list = []
for seed in seed_list:
    pdr_list = []   # index 0: 200nodes  index 1: 300nodes index:2 400nodes
    for node in nodenum_list:
        path = lsgo_dict[seed][node]
        lsgo_df = pd.read_csv(path)
        pdr = lsgo_df['PDR'].mean()
        pdr_list.append(pdr)
    pdr_list.append(seed)
    total_pdr_list.append(pdr_list)

# print(total_pdr_list)
print(col_average(total_pdr_list))

[0.33474371 0.42503952 0.70467172]


In [16]:
# Overheadの平均を計算 SIGO Geocast
total_overhead_list = []
for seed in seed_list:
    overhead_list = []   # index 0: 200nodes  index 1: 300nodes index:2 400nodes
    for node in nodenum_list:
        path = sigo_dict[seed][node]
        sigo_df = pd.read_csv(path)
        mr_recvs = sigo_df['mr_recvs'].sum() #mr内の受信ノード数
        sequence_count = sigo_df[(sigo_df['PDR'] != 0)]['mr_recvs'].count()
        geocast_packets = sigo_df[(sigo_df['PDR'] != 0)]['geocast_packets'].sum()
        flooding_packets = sigo_df[(sigo_df['PDR'] != 0)]['flooding_packets'].sum()
        packet_count = geocast_packets + flooding_packets
        if mr_recvs == 0:
            overhead = 0 #いったん仮置き
            print('seed', seed, 'num', node, 'mr_recvs = 0')
        else:
            overhead = packet_count / mr_recvs
        overhead_list.append(overhead)
    overhead_list.append(seed)
    if 0 not in overhead_list:
        total_overhead_list.append(overhead_list)
    else:
        print('seed', seed, 'list', overhead_list)
print(col_average(total_overhead_list))

[3.3515318  2.53876687 2.37051879]


In [17]:
# Overheadの平均を計算 LSGO Geocast
total_overhead_list = []
for seed in seed_list:
    overhead_list = []   # index 0: 200nodes  index 1: 300nodes index:2 400nodes
    for node in nodenum_list:
        path = lsgo_dict[seed][node]
        lsgo_df = pd.read_csv(path)
        mr_recvs = lsgo_df['mr_recvs'].sum() #mr内の受信ノード数
        sequence_count = lsgo_df[(lsgo_df['PDR'] != 0)]['mr_recvs'].count()
        geocast_packets = lsgo_df[(lsgo_df['PDR'] != 0)]['geocast_packets'].sum()
        flooding_packets = lsgo_df[(lsgo_df['PDR'] != 0)]['flooding_packets'].sum()
        packet_count = geocast_packets + flooding_packets
        if mr_recvs == 0:
            overhead = 0 #いったん仮置き
            print('seed', seed, 'num', node, 'mr_recvs = 0')
        else:
            overhead = packet_count / mr_recvs
        overhead_list.append(overhead)
    overhead_list.append(seed)
    if 0 not in overhead_list:
        total_overhead_list.append(overhead_list)
    else:
        print('seed', seed, 'list', overhead_list)
print(col_average(total_overhead_list))

[3.19361924 2.51472214 2.36219866]


In [15]:
# packet数の平均を計算 SIGO Geocast
total_packet_list = []
for seed in seed_list:
    packet_list = []   # index 0: 200nodes  index 1: 300nodes index:2 400nodes
    for node in nodenum_list:
        path = sigo_dict[seed][node]
        sigo_df = pd.read_csv(path)
        geocast_packets = sigo_df[(sigo_df['PDR'] != 0)]['geocast_packets'].sum()
        flooding_packets = sigo_df[(sigo_df['PDR'] != 0)]['flooding_packets'].sum()
        packet_count = geocast_packets + flooding_packets
        packet_list.append(packet_count)
    packet_list.append(seed)
#     print(packet_list)
    total_packet_list.append(packet_list)
print(col_average(total_packet_list))

[244.38383838 390.66666667 788.71717172]


In [42]:
# packet数の平均を計算 LSGO Geocast
total_packet_list = []
for seed in seed_list:
    packet_list = []   # index 0: 200nodes  index 1: 300nodes index:2 400nodes
    for node in nodenum_list:
        path = lsgo_dict[seed][node]
        lsgo_df = pd.read_csv(path)
        geocast_packets = lsgo_df[(lsgo_df['PDR'] != 0)]['geocast_packets'].sum()
        flooding_packets = lsgo_df[(lsgo_df['PDR'] != 0)]['flooding_packets'].sum()
        packet_count = geocast_packets + flooding_packets
        packet_list.append(packet_count)
    packet_list.append(seed)
#     print(packet_list)
    total_packet_list.append(packet_list)
print(col_average(total_packet_list))

[163.88888889 262.74747475 546.80808081]


In [26]:
# geocast パケット数
total_packet_list = []
for seed in seed_list:
    packet_list = []   # index 0: 200nodes  index 1: 300nodes index:2 400nodes
    for node in nodenum_list:
        path = sigo_dict[seed][node]
        sigo_df = pd.read_csv(path)
        geocast_packets = sigo_df[(sigo_df['PDR'] != 0)]['geocast_packets'].sum()
#         flooding_packets = sigo_df[(sigo_df['PDR'] != 0)]['flooding_packets'].sum()
#         packet_count = geocast_packets + flooding_packets
        packet_list.append(geocast_packets)
    packet_list.append(seed)
    total_packet_list.append(packet_list)
print('sigo geocast packet average' , col_average(total_packet_list))

sigo geocast packet average [108.86868687  80.63636364 152.72727273]


In [28]:
# geocast パケット数
total_packet_list = []
for seed in seed_list:
    packet_list = []   # index 0: 200nodes  index 1: 300nodes index:2 400nodes
    for node in nodenum_list:
        path = lsgo_dict[seed][node]
        lsgo_df = pd.read_csv(path)
        geocast_packets = lsgo_df[(lsgo_df['PDR'] != 0)]['geocast_packets'].sum()
#         flooding_packets = lsgo_df[(sigo_df['PDR'] != 0)]['flooding_packets'].sum()
#         packet_count = geocast_packets + flooding_packets
        packet_list.append(geocast_packets)
    packet_list.append(seed)
    total_packet_list.append(packet_list)
print('sigo geocast packet average' , col_average(total_packet_list))

sigo geocast packet average [ 82.53535354  75.98989899 134.08080808]


In [116]:
test_path = sigo_dict['10001']['200']
test_df = pd.read_csv(test_path)
print(test_df['mr_recvs'].sum()) #mr_recvsの合計

#pdr が 0ではないmr_recvの合計
print(test_df[(test_df['PDR'] != 0)]['mr_recvs'].sum()) 

#pdrが0ではない　もの(packet)だけをカウント
print(test_df[(test_df['PDR'] != 0)]['mr_recvs'].count())

94
94
6
